# 2020-01-21 UT Preliminary Analysis

In [10]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import matplotlib.pyplot as plt
from imaka.analysis import plot_stats
from astropy.table import Table
import numpy as np
from importlib import reload
from imaka.reduce.nights import reduce_2020_01_21 as redu

reload(redu)
reload(plot_stats)

root_dir = "/g/lu/data/imaka/onaga/20200121/sta/"
print("Root directory:",root_dir)
stats_dir = root_dir + "reduce/stats/"
plots_dir = root_dir + "reduce/plots/"
stack_dir = root_dir + "reduce/stacks/"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Root directory: /g/lu/data/imaka/onaga/20200121/sta/


# Data Reduction

Fully reduce the data from 20200121 UT. Don't re-run this unless you really need to as it takes awhile. After you re-run the data reduction, please remember to load it back up to onaga.

In [11]:
"""Set the appropriate root directory for your machine"""
redu.root_dir = root_dir

""""Make sky and flat images and use them to reduce science images"""
#redu.make_sky() #using frames from HR2286 to make sky images
#redu.make_flat()  # Using flat from 20181223
redu.reduce_FLD2()

# """"Find stars in clean images and calculate stats"""
# redu.find_stars_FLD2()
# redu.calc_star_stats()

# """"Make a stack of images for each mode and analyze stacks"""
# redu.stack_FLD2()
# redu.analyze_stacks()

Working on file:  /g/lu/data/imaka/onaga/20200121/sta/Beehive-W/sta003_o.fits
Working on file:  /g/lu/data/imaka/onaga/20200121/sta/Beehive-W/sta004_o.fits
Working on file:  /g/lu/data/imaka/onaga/20200121/sta/Beehive-W/sta005_o.fits
Working on file:  /g/lu/data/imaka/onaga/20200121/sta/Beehive-W/sta007_o.fits
Working on file:  /g/lu/data/imaka/onaga/20200121/sta/Beehive-W/sta009_o.fits
Working on file:  /g/lu/data/imaka/onaga/20200121/sta/Beehive-W/sta011_o.fits
Working on file:  /g/lu/data/imaka/onaga/20200121/sta/Beehive-W/sta012_o.fits
Working on file:  /g/lu/data/imaka/onaga/20200121/sta/Beehive-W/sta013_o.fits
Working on file:  /g/lu/data/imaka/onaga/20200121/sta/Beehive-W/sta014_o.fits
Working on file:  /g/lu/data/imaka/onaga/20200121/sta/Beehive-W/sta015_o.fits
Working on file:  /g/lu/data/imaka/onaga/20200121/sta/Beehive-W/sta016_o.fits
Working on file:  /g/lu/data/imaka/onaga/20200121/sta/Beehive-W/sta017_o.fits
Working on file:  /g/lu/data/imaka/onaga/20200121/sta/Beehive-W/

ValueError: operands could not be broadcast together with shapes (5280,5072) (5280,5280) 

In [12]:
import pdb
pdb.pm()

> /u/caylananderson/code/python/imaka/imaka/reduce/reduce_fli.py(156)clean_single_image()
-> img = img / flat
(Pdb) print(np.shape(img))
(5280, 5072)
(Pdb) print(np.shape(flat))
(5280, 5280)
(Pdb) exit
